# Coupled generative adversarial networks

Ref.: LIU, Ming-Yu; TUZEL, Oncel. Coupled generative adversarial networks. In: Advances in neural information processing systems. 2016. p. 469-477.

In [3]:
import scipy
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

# generator

In [5]:
def build_generators(img_shape, latent_dim):
    """ structure is hard-coded
    """
    
    # Shared weights between generators
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    noise = Input(shape=(latent_dim,))
    feature_repr = model(noise)

    # Generator 1
    g1 = Dense(1024)(feature_repr)
    g1 = LeakyReLU(alpha=0.2)(g1)
    g1 = BatchNormalization(momentum=0.8)(g1)
    g1 = Dense(np.prod(img_shape), activation='tanh')(g1)
    img1 = Reshape(img_shape)(g1)

    # Generator 2
    g2 = Dense(1024)(feature_repr)
    g2 = LeakyReLU(alpha=0.2)(g2)
    g2 = BatchNormalization(momentum=0.8)(g2)
    g2 = Dense(np.prod(img_shape), activation='tanh')(g2)
    img2 = Reshape(img_shape)(g2)

    model.summary()

    return Model(noise, img1), Model(noise, img2)

# discriminator

In [6]:
def build_discriminators(img_shape):

    img1 = Input(shape=img_shape)
    img2 = Input(shape=img_shape)

    # Shared discriminator layers
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    img1_embedding = model(img1)
    img2_embedding = model(img2)

    # Discriminator 1
    validity1 = Dense(1, activation='sigmoid')(img1_embedding)
    # Discriminator 2
    validity2 = Dense(1, activation='sigmoid')(img2_embedding)

    return Model(img1, validity1), Model(img2, validity2)

In [7]:
def sample_images(epoch, g1, g2):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * int(c/2), 100))
    gen_imgs1 = g1.predict(noise)
    gen_imgs2 = g2.predict(noise)

    gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

# train the model

In [8]:
def train(g1, g2, d1, d2, combined,
          epochs, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Images in domain A and B (rotated)
    X1 = X_train[:int(X_train.shape[0]/2)]
    X2 = X_train[int(X_train.shape[0]/2):]
    X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ----------------------
        #  Train Discriminators
        # ----------------------

        # Select a random batch of images
        idx = np.random.randint(0, X1.shape[0], batch_size)
        imgs1 = X1[idx]
        imgs2 = X2[idx]

        # Sample noise as generator input
        noise = np.random.normal(0, 1, (batch_size, 100))

        # Generate a batch of new images
        gen_imgs1 = g1.predict(noise)
        gen_imgs2 = g2.predict(noise)

        # Train the discriminators
        d1_loss_real = d1.train_on_batch(imgs1, valid)
        d2_loss_real = d2.train_on_batch(imgs2, valid)
        d1_loss_fake = d1.train_on_batch(gen_imgs1, fake)
        d2_loss_fake = d2.train_on_batch(gen_imgs2, fake)
        d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
        d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)


        # ------------------
        #  Train Generators
        # ------------------

        g_loss = combined.train_on_batch(noise, [valid, valid])

        # Plot the progress
        print ("%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
            % (epoch, d1_loss[0], 100*d1_loss[1], d2_loss[0], 100*d2_loss[1], g_loss[0]))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch, g1, g2)

# main()

In [9]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [10]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile both discriminator
d1, d2 = build_discriminators(img_shape)

d1.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

d2.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# Build both generator
g1, g2 = build_generators(img_shape, latent_dim)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
Total params: 160,512
Trainable params: 158,976
Non-trainable params: 1,536
____________________________________________

In [13]:
# The generator takes noise as input and generated imgs
z = Input(shape=(latent_dim,))
img1 = g1(z)
img2 = g2(z)

In [14]:
# For the combined model we will only train the generators
d1.trainable = False
d2.trainable = False



In [15]:
# The valid takes generated images as input and determines validity
valid1 = d1(img1)
valid2 = d2(img2)

In [16]:
# The combined model  (stacked generators and discriminators)
# Trains generators to fool discriminators
combined = Model(z, [valid1, valid2])
combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                            optimizer=optimizer)

# Train the model

In [17]:
epochs=3000
# epochs=30000
train(g1, g2, d1, d2, combined,
      epochs=epochs, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D1 loss: 0.918094, acc.: 46.88%] [D2 loss: 0.604709, acc.: 62.50%] [G loss: 1.501780]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D1 loss: 0.397216, acc.: 96.88%] [D2 loss: 0.343117, acc.: 89.06%] [G loss: 1.536288]
2 [D1 loss: 0.325359, acc.: 92.19%] [D2 loss: 0.328741, acc.: 82.81%] [G loss: 1.660351]
3 [D1 loss: 0.303079, acc.: 92.19%] [D2 loss: 0.316144, acc.: 85.94%] [G loss: 1.851377]
4 [D1 loss: 0.281995, acc.: 95.31%] [D2 loss: 0.295434, acc.: 93.75%] [G loss: 2.076351]
5 [D1 loss: 0.265806, acc.: 98.44%] [D2 loss: 0.301856, acc.: 87.50%] [G loss: 2.408001]
6 [D1 loss: 0.222210, acc.: 98.44%] [D2 loss: 0.226912, acc.: 98.44%] [G loss: 2.985777]
7 [D1 loss: 0.170685, acc.: 98.44%] [D2 loss: 0.209495, acc.: 98.44%] [G loss: 3.375984]
8 [D1 loss: 0.156022, acc.: 100.00%] [D2 loss: 0.151447, acc.: 100.00%] [G loss: 3.787433]
9 [D1 loss: 0.114053, acc.: 100.00%] [D2 loss: 0.140640, acc.: 100.00%] [G loss: 4.260783]
10 [D1 loss: 0.087218, acc.: 100.00%] [D2 loss: 0.098014, acc.: 100.00%] [G loss: 4.671067]
11 [D1 loss: 0.081186, acc.: 100.00%] [D2 loss: 0.080641, acc.: 100.00%] [G loss: 4.970495]
12 [D1 loss

92 [D1 loss: 0.114201, acc.: 95.31%] [D2 loss: 0.035227, acc.: 98.44%] [G loss: 11.568077]
93 [D1 loss: 0.170189, acc.: 90.62%] [D2 loss: 0.030626, acc.: 100.00%] [G loss: 13.491845]
94 [D1 loss: 0.792813, acc.: 75.00%] [D2 loss: 1.587731, acc.: 32.81%] [G loss: 7.175697]
95 [D1 loss: 0.266301, acc.: 87.50%] [D2 loss: 0.334540, acc.: 85.94%] [G loss: 7.555627]
96 [D1 loss: 0.155245, acc.: 89.06%] [D2 loss: 0.284816, acc.: 90.62%] [G loss: 10.394873]
97 [D1 loss: 0.035605, acc.: 98.44%] [D2 loss: 0.079459, acc.: 95.31%] [G loss: 10.828533]
98 [D1 loss: 0.103069, acc.: 96.88%] [D2 loss: 0.113216, acc.: 95.31%] [G loss: 11.014616]
99 [D1 loss: 0.017494, acc.: 100.00%] [D2 loss: 0.117127, acc.: 96.88%] [G loss: 10.881680]
100 [D1 loss: 0.045461, acc.: 100.00%] [D2 loss: 0.061805, acc.: 96.88%] [G loss: 10.304309]
101 [D1 loss: 0.044672, acc.: 100.00%] [D2 loss: 0.063246, acc.: 96.88%] [G loss: 10.617430]
102 [D1 loss: 0.016717, acc.: 100.00%] [D2 loss: 0.036694, acc.: 100.00%] [G loss: 10.

...

2563 [D1 loss: 0.599116, acc.: 68.75%] [D2 loss: 0.658954, acc.: 57.81%] [G loss: 2.208485]
2564 [D1 loss: 0.542651, acc.: 76.56%] [D2 loss: 0.570567, acc.: 78.12%] [G loss: 2.260282]
2565 [D1 loss: 0.579524, acc.: 71.88%] [D2 loss: 0.616892, acc.: 65.62%] [G loss: 2.100487]
2566 [D1 loss: 0.659957, acc.: 60.94%] [D2 loss: 0.724467, acc.: 45.31%] [G loss: 1.902709]
2567 [D1 loss: 0.614744, acc.: 59.38%] [D2 loss: 0.600556, acc.: 60.94%] [G loss: 2.121902]
2568 [D1 loss: 0.539726, acc.: 71.88%] [D2 loss: 0.645741, acc.: 67.19%] [G loss: 2.103993]
2569 [D1 loss: 0.554314, acc.: 68.75%] [D2 loss: 0.566299, acc.: 67.19%] [G loss: 2.151618]
2570 [D1 loss: 0.604937, acc.: 70.31%] [D2 loss: 0.606509, acc.: 62.50%] [G loss: 2.145060]
2571 [D1 loss: 0.686428, acc.: 56.25%] [D2 loss: 0.609413, acc.: 68.75%] [G loss: 2.202036]
2572 [D1 loss: 0.629400, acc.: 68.75%] [D2 loss: 0.584613, acc.: 73.44%] [G loss: 2.163031]
2573 [D1 loss: 0.646721, acc.: 64.06%] [D2 loss: 0.577617, acc.: 67.19%] [G loss

2654 [D1 loss: 0.626499, acc.: 60.94%] [D2 loss: 0.577058, acc.: 67.19%] [G loss: 2.054017]
2655 [D1 loss: 0.593964, acc.: 71.88%] [D2 loss: 0.583492, acc.: 70.31%] [G loss: 2.120618]
2656 [D1 loss: 0.524036, acc.: 76.56%] [D2 loss: 0.559413, acc.: 67.19%] [G loss: 2.205699]
2657 [D1 loss: 0.678203, acc.: 54.69%] [D2 loss: 0.541686, acc.: 81.25%] [G loss: 2.048382]
2658 [D1 loss: 0.619763, acc.: 65.62%] [D2 loss: 0.601833, acc.: 68.75%] [G loss: 2.075934]
2659 [D1 loss: 0.636739, acc.: 62.50%] [D2 loss: 0.612129, acc.: 65.62%] [G loss: 2.153712]
2660 [D1 loss: 0.600706, acc.: 65.62%] [D2 loss: 0.566644, acc.: 68.75%] [G loss: 2.275991]
2661 [D1 loss: 0.598788, acc.: 68.75%] [D2 loss: 0.640275, acc.: 64.06%] [G loss: 2.130232]
2662 [D1 loss: 0.611602, acc.: 60.94%] [D2 loss: 0.598131, acc.: 68.75%] [G loss: 2.163229]
2663 [D1 loss: 0.676569, acc.: 54.69%] [D2 loss: 0.608010, acc.: 65.62%] [G loss: 2.107105]
2664 [D1 loss: 0.650744, acc.: 60.94%] [D2 loss: 0.560749, acc.: 73.44%] [G loss

2744 [D1 loss: 0.582542, acc.: 67.19%] [D2 loss: 0.554435, acc.: 71.88%] [G loss: 2.239780]
2745 [D1 loss: 0.587080, acc.: 65.62%] [D2 loss: 0.571571, acc.: 75.00%] [G loss: 2.145100]
2746 [D1 loss: 0.647244, acc.: 60.94%] [D2 loss: 0.694340, acc.: 54.69%] [G loss: 1.992785]
2747 [D1 loss: 0.661346, acc.: 59.38%] [D2 loss: 0.567010, acc.: 62.50%] [G loss: 1.994797]
2748 [D1 loss: 0.616533, acc.: 71.88%] [D2 loss: 0.688739, acc.: 53.12%] [G loss: 1.993128]
2749 [D1 loss: 0.640014, acc.: 57.81%] [D2 loss: 0.566960, acc.: 68.75%] [G loss: 2.323874]
2750 [D1 loss: 0.671827, acc.: 59.38%] [D2 loss: 0.589548, acc.: 68.75%] [G loss: 2.316144]
2751 [D1 loss: 0.586288, acc.: 70.31%] [D2 loss: 0.593766, acc.: 73.44%] [G loss: 2.204543]
2752 [D1 loss: 0.652534, acc.: 65.62%] [D2 loss: 0.553505, acc.: 78.12%] [G loss: 2.147875]
2753 [D1 loss: 0.660371, acc.: 56.25%] [D2 loss: 0.579117, acc.: 73.44%] [G loss: 2.211433]
2754 [D1 loss: 0.604002, acc.: 67.19%] [D2 loss: 0.549423, acc.: 70.31%] [G loss

2834 [D1 loss: 0.573373, acc.: 70.31%] [D2 loss: 0.535937, acc.: 68.75%] [G loss: 2.113077]
2835 [D1 loss: 0.562268, acc.: 76.56%] [D2 loss: 0.540281, acc.: 78.12%] [G loss: 2.130850]
2836 [D1 loss: 0.560807, acc.: 68.75%] [D2 loss: 0.588879, acc.: 65.62%] [G loss: 2.194700]
2837 [D1 loss: 0.603334, acc.: 73.44%] [D2 loss: 0.601352, acc.: 70.31%] [G loss: 2.141674]
2838 [D1 loss: 0.553279, acc.: 75.00%] [D2 loss: 0.610494, acc.: 57.81%] [G loss: 2.193762]
2839 [D1 loss: 0.629007, acc.: 68.75%] [D2 loss: 0.533021, acc.: 79.69%] [G loss: 2.139055]
2840 [D1 loss: 0.601493, acc.: 65.62%] [D2 loss: 0.538438, acc.: 68.75%] [G loss: 2.149949]
2841 [D1 loss: 0.585421, acc.: 67.19%] [D2 loss: 0.527996, acc.: 81.25%] [G loss: 2.205391]
2842 [D1 loss: 0.514211, acc.: 78.12%] [D2 loss: 0.609629, acc.: 65.62%] [G loss: 2.289531]
2843 [D1 loss: 0.594687, acc.: 68.75%] [D2 loss: 0.594953, acc.: 65.62%] [G loss: 2.235615]
2844 [D1 loss: 0.608859, acc.: 70.31%] [D2 loss: 0.534546, acc.: 76.56%] [G loss

2925 [D1 loss: 0.523800, acc.: 78.12%] [D2 loss: 0.587507, acc.: 71.88%] [G loss: 2.192953]
2926 [D1 loss: 0.581352, acc.: 70.31%] [D2 loss: 0.666334, acc.: 65.62%] [G loss: 2.082850]
2927 [D1 loss: 0.633006, acc.: 56.25%] [D2 loss: 0.638029, acc.: 64.06%] [G loss: 2.104582]
2928 [D1 loss: 0.567604, acc.: 71.88%] [D2 loss: 0.622596, acc.: 68.75%] [G loss: 2.086561]
2929 [D1 loss: 0.578668, acc.: 67.19%] [D2 loss: 0.543243, acc.: 67.19%] [G loss: 2.191237]
2930 [D1 loss: 0.621013, acc.: 65.62%] [D2 loss: 0.625292, acc.: 65.62%] [G loss: 2.135205]
2931 [D1 loss: 0.689648, acc.: 53.12%] [D2 loss: 0.596191, acc.: 70.31%] [G loss: 2.069831]
2932 [D1 loss: 0.557140, acc.: 78.12%] [D2 loss: 0.609199, acc.: 67.19%] [G loss: 2.217280]
2933 [D1 loss: 0.559464, acc.: 67.19%] [D2 loss: 0.549993, acc.: 78.12%] [G loss: 2.248778]
2934 [D1 loss: 0.563416, acc.: 73.44%] [D2 loss: 0.593040, acc.: 65.62%] [G loss: 2.186382]
2935 [D1 loss: 0.667106, acc.: 62.50%] [D2 loss: 0.615167, acc.: 68.75%] [G loss